In [ ]:
# === Install Dependencies (for Colab/Linux) ===
!apt-get update
!apt-get install -y imagemagick fontconfig ttf-mscorefonts-installer fonts-dejavu
!fc-cache -fv
!sed -i 's/rights="none"/rights="read|write"/g' /etc/ImageMagick*/policy.xml
!pip install gTTS pydub diffusers transformers accelerate scipy torchaudio moviepy gradio
!apt install ffmpeg -y

# === Imports ===
import os, gc, torch, requests, datetime
from transformers import CLIPTokenizer
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from gtts import gTTS
from pydub import AudioSegment
from moviepy.editor import *
import gradio as gr

# === Constants ===
os.environ["IMAGEMAGICK_BINARY"] = "/usr/bin/convert"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

GROQ_API_KEY = "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"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,801 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,750 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

  if event.key is 'enter':



In [ ]:
# === Load Tokenizer ===
clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# === Memory Management ===
def free_cuda():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

# === Load Models ===
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to(device)
upscale_pipe = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to(device)

# === Helper Functions ===
def clean_output(text):
    return text.encode("utf-16", "surrogatepass").decode("utf-16", "ignore")

def fallback_story(topic):
    title = f"Why {topic.capitalize()} Matters"
    story = f"{topic.capitalize()} is shaping the world. Let’s explore how it’s transforming our lives and future."
    return title, story

def clip_truncate(prompt, max_tokens=77):
    tokens = clip_tokenizer.tokenize(prompt)
    return clip_tokenizer.convert_tokens_to_string(tokens[:max_tokens])

def generate_story_llama3(topic, word_length=900):
    prompt = (
        f"Write a compelling, engaging LinkedIn-style social media post about the topic: '{topic}'.\n"
        f"Limit the story to approximately {word_length} words.\n"
        f"Include a title and the main story. Return in format:\nTitle: <title>\nStory: <story>"
    )
    try:
        res = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            json={"model": "llama3-70b-8192", "messages": [{"role": "user", "content": prompt}]},
            headers={"Authorization": f"Bearer {GROQ_API_KEY}"}
        )
        output = res.json()['choices'][0]['message']['content']
        if "Title:" in output and "Story:" in output:
            title = output.split("Title:")[1].split("\n")[0].strip()
            story = output.split("Story:")[1].strip()
            os.makedirs("post", exist_ok=True)
            with open("post/story.txt", "w", encoding="utf-8") as f:
                f.write(f"Title: {title}\n\nStory: {story}")
            return title, story
        else:
            return fallback_story(topic)
    except Exception as e:
        print(f"[Groq Error] {e}")
        return fallback_story(topic)


tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, du

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, du

In [ ]:
def generate_hashtags(topic):
    prompt = f"Generate 5 trending hashtags for: '{topic}'. Return in a line."
    try:
        res = requests.post("https://api.groq.com/openai/v1/chat/completions",
            json={"model": "llama3-70b-8192", "messages": [{"role": "user", "content": prompt}]},
            headers={"Authorization": f"Bearer {GROQ_API_KEY}"})
        return clean_output(res.json()['choices'][0]['message']['content']).strip()
    except:
        return "#AI #Tech #Innovation #Future #Leadership"

def generate_video_hook(topic):
    prompt = f"Generate a viral short video hook (max 12 words) for: '{topic}'"
    try:
        res = requests.post("https://api.groq.com/openai/v1/chat/completions",
            json={"model": "llama3-70b-8192", "messages": [{"role": "user", "content": prompt}]},
            headers={"Authorization": f"Bearer {GROQ_API_KEY}"})
        return clean_output(res.json()['choices'][0]['message']['content']).strip()
    except:
        return "This will change how you see everything!"

def generate_image_prompt_from_topic(topic):
    prompt = (
        f"You're an expert visual storyteller. Create a vivid, imaginative prompt for Stable Diffusion that visualizes the topic: '{topic}'.\n"
        "Avoid distortions, unrelated animals, or fantastical elements unless explicitly described. The prompt should generate Dslr clicked -style visuals."
    )
    try:
        res = requests.post("https://api.groq.com/openai/v1/chat/completions",
            json={"model": "llama3-70b-8192", "messages": [{"role": "user", "content": prompt}]},
            headers={"Authorization": f"Bearer {GROQ_API_KEY}"})
        return clip_truncate(clean_output(res.json()['choices'][0]['message']['content']).strip())
    except:
        return clip_truncate(f"A cinematic illustration of: {topic}, in animated style")

def generate_google_tts_audio(text, out_path_wav, min_duration_sec=60):
    tmp_mp3 = "tmp_gtts.mp3"
    gTTS(text=text, lang='en').save(tmp_mp3)
    audio = AudioSegment.from_mp3(tmp_mp3)
    if len(audio) < min_duration_sec * 1000:
        audio += AudioSegment.silent(duration=(min_duration_sec * 1000 - len(audio)))
    audio.export(out_path_wav, format="wav")
    os.remove(tmp_mp3)

In [ ]:
def generate_images_with_custom_prompt(prompt, base_name):
    image_paths = []
    generator = torch.Generator(device=device).manual_seed(42)
    for i in range(5):
        img = pipe(prompt=prompt, guidance_scale=8.0, generator=generator).images[0]
        img = img.resize((1080, 1080))
        img = upscale_pipe(prompt=prompt, image=img, strength=0.6, guidance_scale=8.0, generator=generator).images[0]
        path = f"{base_name}_img_{i}.png"
        img.save(path)
        image_paths.append(path)
        free_cuda()
    return image_paths

def ken_burns_zoom_with_captions(image_path, duration, top_text="", zoom=1.1):
    background = ColorClip(size=(1080, 1920), color=(0, 0, 0), duration=duration)
    image_clip = ImageClip(image_path).resize(height=1440).set_duration(duration)

    def zoom_in_out(get_frame, t):
        frame = get_frame(t)
        factor = 1 + (zoom - 1) * (t / duration)
        h, w, _ = frame.shape
        new_w, new_h = int(w / factor), int(h / factor)
        x1, y1 = (w - new_w) // 2, (h - new_h) // 2
        return frame[y1:y1+new_h, x1:x1+new_w]

    zoomed = image_clip.fl(zoom_in_out).set_position((0, 240))
    caption = TextClip("\n".join(top_text.split(" ", 6)), fontsize=60, color="white", font="DejaVu-Serif-Bold", method='label', align='center', size=(1080, None))
    caption = caption.set_position(("center", 180)).set_duration(duration)

    return CompositeVideoClip([background, zoomed, caption])

def process_pipeline(topic, output_type, word_length):
    os.makedirs("post", exist_ok=True)
    title, story = generate_story_llama3(topic, word_length=word_length)
    prompt = generate_image_prompt_from_topic(topic)
    hashtags = generate_hashtags(topic)
    hook = generate_video_hook(topic)

    result = f"""📌 **Topic**: {topic}\n🎬 **Output Type**: {output_type}\n\n📝 **Title**: {title}\n📖 **Story**: {story}\n\n📣 **Hook**: {hook}\n🏷️ **Hashtags**: {hashtags}"""

    out_path = None
    if output_type in ["Image", "Text Image Video", "Video"]:
        images = generate_images_with_custom_prompt(prompt, base_name="post/image")

    if output_type in ["Video", "Text Image Video"]:
        generate_google_tts_audio(story, out_path_wav="post/audio.wav")
        duration_per_image = 12  # 12 sec per image for 5 images = 60 sec
        clips = [ken_burns_zoom_with_captions(img, duration=duration_per_image, top_text=hook) for img in images]
        final_clip = concatenate_videoclips(clips).set_audio(AudioFileClip("post/audio.wav"))
        out_path = f"post/final_video_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.mp4"
        final_clip.write_videofile(out_path, fps=24, codec="libx264", audio_codec="aac")

    return result, out_path


In [ ]:
# === UI Code ===
custom_css = """
#title h2 {
  color: #ff4b4b !important;
  font-size: 28px !important;
  font-weight: 800;
}
.gr-button {
  background-color: #ff4b4b !important;
  color: white !important;
  font-weight: bold !important;
  border-radius: 8px;
}
.gr-button:hover {
  background-color: #e43e3e !important;
}
.gr-box, .gr-textbox, .gr-dropdown, .gr-image, .gr-accordion, .gr-file {
  background-color: rgba(0, 0, 0, 0.6) !important;
  border-radius: 12px;
  color: white !important;
}
"""

background_html = """
<style>
html, body, .gradio-container {
  background-image: url('https://i.ibb.co/8WG4fZS/vibrant-colors-flow-abstract-wave-pattern-generated-by-ai.jpg');
  background-size: cover;
  background-position: center;
  background-attachment: fixed;
  font-family: 'Segoe UI', sans-serif;
}
</style>
"""

def run_gradio():
    with gr.Blocks(css=custom_css) as demo:
        gr.HTML(background_html)
        gr.Markdown("## 🎬 AI Social Media Post Generator <small>(Groq + TTS + Diffusion)</small>", elem_id="title")
        with gr.Row():
            with gr.Column():
                topic = gr.Textbox(label="📌 Topic", placeholder="e.g., AI in Indian Education")
                output_type = gr.Radio(choices=["Text", "Image", "Video", "Text Image Video"], label="🎞️ Output Type")
                word_length = gr.Slider(label="✏️ Desired Story Length (words)", minimum=500, maximum=1500, step=100, value=900, visible=False)
                def toggle_slider(output_type):
                    return gr.update(visible=output_type in ["Text", "Text Image Video"])
                output_type.change(fn=toggle_slider, inputs=[output_type], outputs=[word_length])
                generate_btn = gr.Button("🚀 Generate Post")
            with gr.Column():
                image_preview = gr.Image(label="🖼️ Preview (Image or Video)", show_label=True, interactive=False)
                result = gr.Textbox(label="📜 Generated Summary", lines=14)
                download = gr.File(label="⬇️ Download Video")
        generate_btn.click(fn=process_pipeline, inputs=[topic, output_type, word_length], outputs=[result, download])
    demo.launch(debug=True, share=True)

if __name__ == "__main__":
    run_gradio()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a9896011e6b881d85a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
